In [4]:
!pip install keras-tuner
!pip install prophet
!pip install pmdarima

     |████████████████████████████████| 98 kB 3.2 MB/s 
     |████████████████████████████████| 65 kB 2.3 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=6640438 sha256=01485120f029867486635ef8c6ecd9877a5e50c8fafeacae6ec0f83a684098e0
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.
     |████████████████████████████████| 1.4 MB 4.3 MB

In [1]:
import shutil
import os
if "deepant" in os.listdir(os.curdir):
    shutil.rmtree("deepant")
if "untitled_project" in os.listdir(os.curdir):
    shutil.rmtree("untitled_project")
if "lstm" in os.listdir(os.curdir):
    shutil.rmtree("lstm")
if "gru" in os.listdir(os.curdir):
    shutil.rmtree("gru")

if "tesi_magistrale" in os.listdir(os.curdir):
    shutil.rmtree("tesi_magistrale")
!git clone https://github.com/salvalai/tesi_magistrale.git

import sys
if "./tesi_magistrale" not in sys.path:
    sys.path.append("./tesi_magistrale")

Cloning into 'tesi_magistrale'...
remote: Enumerating objects: 974, done.
remote: Counting objects: 100% (974/974), done.
remote: Compressing objects: 100% (614/614), done.
remote: Total 974 (delta 569), reused 705 (delta 308), pack-reused 0
Receiving objects: 100% (974/974), 36.67 MiB | 12.63 MiB/s, done.
Resolving deltas: 100% (569/569), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
tsdir = "/content/drive/MyDrive/magistrale/tesi/timeseries"
print(f"@{tsdir}: {os.listdir(tsdir)}")
groups = [(g, os.path.join(tsdir, g)) 
            for g in os.listdir(tsdir) 
            if os.path.isdir(os.path.join(tsdir, g))
        ]
groups

Mounted at /content/drive
@/content/drive/MyDrive/magistrale/tesi/timeseries: ['training.ipynb', 'timeseries.zip', 'conversionTime.csv', 's1', 's2', 's3', 'repr_tsIDs.txt', 's11', 's10', 's9', 's8', 's4', 's5', 's6', 's7', 'hyperparams', 'extraction_selection.ipynb', 'visualization.ipynb']


[('s1', '/content/drive/MyDrive/magistrale/tesi/timeseries/s1'),
 ('s2', '/content/drive/MyDrive/magistrale/tesi/timeseries/s2'),
 ('s3', '/content/drive/MyDrive/magistrale/tesi/timeseries/s3'),
 ('s11', '/content/drive/MyDrive/magistrale/tesi/timeseries/s11'),
 ('s10', '/content/drive/MyDrive/magistrale/tesi/timeseries/s10'),
 ('s9', '/content/drive/MyDrive/magistrale/tesi/timeseries/s9'),
 ('s8', '/content/drive/MyDrive/magistrale/tesi/timeseries/s8'),
 ('s4', '/content/drive/MyDrive/magistrale/tesi/timeseries/s4'),
 ('s5', '/content/drive/MyDrive/magistrale/tesi/timeseries/s5'),
 ('s6', '/content/drive/MyDrive/magistrale/tesi/timeseries/s6'),
 ('s7', '/content/drive/MyDrive/magistrale/tesi/timeseries/s7'),
 ('hyperparams',
  '/content/drive/MyDrive/magistrale/tesi/timeseries/hyperparams')]

In [8]:
import json
import time

import pandas as pd

from ml_microservice.anomaly_detection import transformers
from ml_microservice.anomaly_detection import tuners
from ml_microservice.anomaly_detection import evaluators

pred_ext = "{}_prediction"

def tune(print_info, sname, ts: pd.DataFrame, path):
    result = {}
    
    pred_folder = os.path.join(path, pred_ext.format(sname))
    if not os.path.exists(pred_folder):
        os.makedirs(pred_folder)

    models = [
        ('wingauss', tuners.WindGaussTuner()),
        #('deepant', tuners.DeepAnTTuner()),
        #('gru', tuners.GRUTuner()),
        #('lstm', tuners.LSTMTuner()),
        #('sarimax', tuners.SARIMAXTuner()),
        #('prophet', tuners.ProphetTuner()),
    ]
    
    for t_name, t in models:
        pred_file = os.path.join(pred_folder, f"{t_name}.csv")
        #if os.path.exists(pred_file):
        #    continue
        result[t_name] = {}
        print(f"{print_info} Tuner: {t_name}")

        preproc = transformers.Preprocessor(ts.copy(deep = True))
        train, test = preproc.train_test

        t0 = time.time()
        best_model = t.tune(train).best_model_
        result[t_name]["tune_total_time"] = time.time() - t0
        print(f"{print_info} Tuner {t_name}, time: {result[t_name]['tune_total_time']}")
        result[t_name]["best_config"] = best_model.get_params()
        
        print(f"{print_info} Evaluating best {t_name}")
        ev = evaluators.GPEvaluator()
        scores = ev.evaluate(best_model, test)
        ev.prediction_.to_csv(
            pred_file, 
            index = False
        )
        result[t_name]["scores"] = scores
        best_params_file = f"{t_name}_hypertuning.json"
        print(f"{print_info} Evaluating best {t_name}: done")
        with open(os.path.join(pred_folder, best_params_file), "w") as fout:
            json.dump(result[t_name], fout, indent = 4)
    return result

In [9]:
import json
import os
import re
import yaml

import pandas as pd

serie_csv = ".+.csv"
repr_ext = "{:s}_repr_tsIDs.txt"
cluster_ext = "{}_best_cluster.json"

def evaluate(group, path):
    todo = [(s[:-4], s, repr_ext.format(s[:-4])) 
                for s in os.listdir(path) 
                if re.match(serie_csv, s) is not None]
    ret_dict = {}
    ret_dict["group"] = group

    for name, serie, reprs in todo:
        ret_dict[name] = {}
        with open(os.path.join(path, reprs), "r") as fin:
            r = yaml.load(fin)
        if "KMeans" in r.keys():
            clusters = [(c["Closest column"], int(c["Dimension"])) 
                        for k, c in r["KMeans"]["Best silhouette"]["Clusters"].items()]
            best_cluster = max(clusters, key = lambda t : t[1])
        else:
            best_cluster = (name, 1)
        print(f"[{group}-{name}] Best cluster: {best_cluster}")
        ret_dict[name]["best_cluster"] = {
            "id": best_cluster[0],
            "size": best_cluster[1]
        }
        
        cluster_f = cluster_ext.format(name)
        r_file = os.path.join(path, cluster_f)
        with open(r_file, "w") as fout:
            json.dump(ret_dict[name]["best_cluster"], fout, indent = 4)

        ts = pd.DataFrame()
        s = pd.read_csv(os.path.join(path, serie))
        ts["timestamp"] = s["Date"]
        ts["value"] = s[best_cluster[0]]
        results = tune(f"[{group}-{name}]", name, ts, path)
        
        print(f"[{group}-{name}] Results: {results}")
        ret_dict[name]["results"] = results
    return ret_dict

In [10]:
import json
import multiprocessing as mp
import os

try:

    with mp.Pool(processes = 4) as pool:
        r = pool.starmap(evaluate, groups)
    print(json.dumps(r, indent = 4))

except Exception as e:
    print("Catched")
    pool.join()
    if "untitled_project" in os.listdir():
        shutil.rmtree("./untitled_project")
    raise e

[s1-somma_progressiva] Best cluster: ('1', 2)
[s11-net_amount] Best cluster: ('net_amount', 1)
[s2-SC_da_Media_Settimanale_Venduto] Best cluster: ('4', 2)
[s1-somma_progressiva] Tuner: wingauss
[s11-net_amount] Tuner: wingauss
[s2-SC_da_Media_Settimanale_Venduto] Tuner: wingauss
[s11-net_amount] Tuner wingauss, time: 0.016058921813964844
[s1-somma_progressiva] Tuner wingauss, time: 0.03745269775390625
[s11-net_amount] Evaluating best wingauss
[s1-somma_progressiva] Evaluating best wingauss
[s2-SC_da_Media_Settimanale_Venduto] Tuner wingauss, time: 0.024428367614746094
[s2-SC_da_Media_Settimanale_Venduto] Evaluating best wingauss
[s1-somma_progressiva] Evaluating best wingauss: done
[s2-SC_da_Media_Settimanale_Venduto] Evaluating best wingauss: done
[s11-net_amount] Evaluating best wingauss: done
[s1-somma_progressiva] Results: {'wingauss': {'tune_total_time': 0.03745269775390625, 'best_config': {'step': 16, 'w': 32}, 'scores': {'f1': nan, 'precision': nan, 'recall': nan, 'mse': nan, 'r